In [1]:
import tifffile
import numpy as np
import matplotlib.pyplot as plt
import os
from csbdeep.utils import normalize
from stardist.models import StarDist2D
from stardist.plot import render_label

In [2]:
image_paths = sorted(["data/images/"+path for path in os.listdir("data/images") if not path.startswith(".")])
image_paths

['data/images/CTRL_1_M1_3R__0004.tif',
 'data/images/CTRL_4_M6_3L__0008.tif',
 'data/images/E2+DHT_1_M13_3L_0001.tif',
 'data/images/E2+DHT_2_M10_3R_0002.tif',
 'data/images/E2+P4+DHT_1_M7_3L_0013.tif',
 'data/images/E2+P4+DHT_1_M7_3L_0019.tif',
 'data/images/E2+P4_2_M9_3R_0003.tif',
 'data/images/E2+P4_3_M12_3R_0002.tif',
 'data/images/E2_1_M8_3L_0011.tif',
 'data/images/E2_2_M11_4R_0001.tif']

In [3]:
mask_paths = sorted(["data/masks/"+path for path in os.listdir("data/masks") if not path.startswith(".")])
mask_paths

['data/masks/CTRL_1_M1_3R__0004_mask.tif',
 'data/masks/CTRL_4_M6_3L__0008_mask.ome.tif',
 'data/masks/E2+DHT_1_M13_3L_0001_mask.ome.tif',
 'data/masks/E2+DHT_2_M10_3R_0002_mask.ome.tif',
 'data/masks/E2+P4+DHT_1_M7_3L_0013_mask.ome.tif',
 'data/masks/E2+P4+DHT_1_M7_3L_0019_mask.ome.tif',
 'data/masks/E2+P4_2_M9_3R_0003_mask.ome.tif',
 'data/masks/E2+P4_3_M12_3R_0002_mask.ome.tif',
 'data/masks/E2_1_M8_3L_0011_mask.ome.tif',
 'data/masks/E2_2_M11_4R_0001_mask.ome.tif']

In [34]:
# images = list(map(tifffile.imread, image_paths))
# masks = list(map(tifffile.imread, mask_paths))
# data = list(zip(images, masks))

# plt.figure(figsize=(20, 10))

# plt.subplot(1, 2, 1)
# plt.imshow(data[0][0])
# plt.axis("off")
# plt.title("Image")

# plt.subplot(1, 2, 2)
# plt.imshow(data[0][1])
# plt.axis("off")
# plt.title("mask")

In [24]:
test_image = data[0][0]
test_mask = data[0][1]

In [25]:
print("Mask dtype:", test_mask.dtype)
print("Unique values in mask:", np.unique(test_mask))

Mask dtype: uint8
Unique values in mask: [0 1]


In [26]:
# Get sizes of all objects
from skimage import measure
labels = measure.label(test_mask)
props = measure.regionprops(labels)
sizes = [prop.area for prop in props]
print("\nObject sizes:", sorted(sizes))


Object sizes: [1.0, 1.0, 2.0, 3.0, 3.0, 3.0, 4.0, 4.0, 4.0, 4.0, 5.0, 5.0, 5.0, 6.0, 6.0, 6.0, 6.0, 7.0, 7.0, 7.0, 7.0, 8.0, 8.0, 8.0, 9.0, 9.0, 9.0, 9.0, 9.0, 10.0, 10.0, 10.0, 10.0, 10.0, 11.0, 11.0, 11.0, 12.0, 12.0, 14.0, 14.0, 15.0, 15.0, 16.0, 18.0, 18.0, 18.0, 20.0, 20.0, 20.0, 20.0, 21.0, 21.0, 28.0, 31.0, 35.0, 37.0, 43.0, 45.0, 79.0, 163.0, 285.0, 374.0, 5073.0]


In [35]:


# from skimage.morphology import remove_small_objects, remove_small_holes

# cleaned_test_mask = remove_small_objects(test_mask.astype(bool), min_size=200)
# cleaned_test_mask = remove_small_holes(cleaned_test_mask, area_threshold=10)

# plt.figure(figsize=(20, 10))

# plt.subplot(1, 2, 1)
# plt.imshow(test_mask)
# plt.axis("off")
# plt.title("Mask before cleaning")

# plt.subplot(1, 2, 2)
# plt.imshow(cleaned_test_mask)
# plt.axis("off")
# plt.title("Mask after cleaning")